In [1]:
import numpy as np
import pandas as pd
import do_mpc
from Data_Cleaner import data_loader_main
from SINDy import SINDy_Dynamics
from MPC_Controller import Flow_Dynamics_Model, MPC_Controller
from utilis_2D import MPC_params, setup_mpc
from pydmd import DMDc
from utilis_2D import params
import scipy

C:\Users\weizi\anaconda3\envs\SUMO\lib\site-packages\do_mpc\sysid\__init__.py:15: UserWarning: The ONNX feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The ONNX feature is not available. Please install the full version of do-mpc to access this feature.')
C:\Users\weizi\anaconda3\envs\SUMO\lib\site-packages\do_mpc\opcua\__init__.py:14: UserWarning: The opcua feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The opcua feature is not available. Please install the full version of do-mpc to access this feature.')


In [2]:
burnin_step = params['burnin_step']
control_interval = params['control_interval']
burnin_control_step = int(burnin_step/control_interval)

In [3]:
flow_all, _, occupancy_all, flow_dt, _, occupancy_dt = data_loader_main(csv_dict='Sim_Results/Ramp_ALIANA')
control_input = pd.read_csv('Results/Meter_Rate_ALIANA.csv')
control_input = control_input/10

occupancy_all, occupancy_dt = occupancy_all.iloc[burnin_control_step:, :], occupancy_dt.iloc[burnin_control_step:, :]
control_input = control_input.iloc[burnin_control_step:, :]

# occupancy_all, occupancy_dt = occupancy_all.iloc[:-1, :].reset_index(drop=True), occupancy_dt.iloc[:-1, :].reset_index(drop=True)
control_input = control_input.iloc[1:,:].reset_index(drop=True)


In [4]:
occupancy_all = np.array(occupancy_all)
control_input = np.array(control_input)

In [6]:
dmdc = DMDc(svd_rank=0)
dmdc.fit(X=occupancy_all.T, I=control_input.T)

In [19]:
B= dmdc.B

In [20]:
eigs = np.power(
            dmdc.eigs, dmdc.dmd_time["dt"] // dmdc.original_time["dt"]
        )

In [21]:
A = np.linalg.multi_dot(
            [dmdc.modes, np.diag(eigs), np.linalg.pinv(dmdc.modes)]
        )

In [22]:
A

array([[ 1.11655779e-02, -6.68476395e-02, -6.61124734e-02,
         3.10762157e-03, -2.13686108e-02, -2.46564357e-02,
        -1.63357310e-02,  1.34194407e-02],
       [-2.82601314e-02,  7.59238670e-01,  1.40318513e-01,
        -2.42762680e-03,  3.75598601e-01,  6.40321892e-02,
         1.90921556e-01, -4.31585510e-02],
       [-6.78790176e-02,  3.79375097e-01,  4.03154905e-01,
        -1.91411398e-02,  1.15187421e-01,  1.49819667e-01,
         9.24623758e-02, -8.11600522e-02],
       [ 3.46323787e-03, -1.52964172e-02, -2.07551110e-02,
         1.01400762e-03, -3.66489571e-03, -7.63271921e-03,
        -3.68840307e-03,  4.07759128e-03],
       [-3.42411809e-04,  3.23564543e-01, -1.26915499e-02,
         2.86772591e-03,  1.75847463e-01,  1.64247836e-03,
         8.20042654e-02, -5.42124474e-03],
       [-2.45500583e-02,  1.48606346e-01,  1.45288817e-01,
        -6.81782087e-03,  4.78700888e-02,  5.42172567e-02,
         3.63301618e-02, -2.95310330e-02],
       [-6.55387394e-03,  1.888133

In [23]:
B

array([[ 9.76462905e-03, -5.67243314e-03, -6.84165915e-03,
         9.47669903e-04, -1.42811647e-04, -1.37611203e-01],
       [-6.00677657e-02,  8.45415209e-03, -2.19757301e-02,
        -1.61357539e-02, -6.32864050e-04,  6.10114650e-01],
       [ 2.15887926e-01, -6.10772259e-02,  4.61698117e-03,
         4.60295188e-02,  5.20115669e-04, -2.46722644e+00],
       [ 4.68914896e-03, -3.44547434e-03, -5.03355732e-03,
         1.73866567e-04, -1.09821545e-04, -7.25342425e-02],
       [-1.39204609e-02, -1.61026826e-03, -1.37413822e-02,
        -5.13074109e-03, -3.50703150e-04,  1.09475757e-01],
       [-1.95756430e-02, -1.56000815e-03, -1.76170293e-02,
        -6.94052331e-03, -4.52909364e-04,  1.60248775e-01],
       [-1.51125180e-02, -4.45602721e-03, -2.14790643e-02,
        -6.62560071e-03, -5.35522607e-04,  9.46386923e-02],
       [ 5.60018154e-03, -5.17949646e-03, -8.59099241e-03,
        -2.07329836e-04, -1.92014095e-04, -9.61456428e-02]])